In [85]:
# Attempt to generalize Conway's nimbers to characteristic p>2
'''
At each stage, we identify p^(p^k) with a root of the polynomial x^p - x - p^(p^(k-1))
This makes $\\mathbb{N}$ into a field isomorphic to the union of GF(p^(p^k)) over all k
'''

def deg(N : int, p : int = 2) -> int:
    '''
    Returns the largest D such that 
    p ** (p ** D) <= N
    under the p_nimber multiplication, the ordinal p^(p^D)
    is a field extension of Z/pZ of degree p^D
    '''
    D = 0
    if N < p:
        return 0
    while N // (p ** (p ** D)) != 0 :
        D += 1
    return D-1

def p_decomp(N : int, p : int = 2,  as_array = False, degree : int = -1) -> list:
    '''
    returns dict {'prime':p,'degree':degree,0:a_0,1:a_1,...,p-1:a_{p-1}} in the decomposition
    N = sum_{k=0}^{p-1} a_k * p^(k*(p^L)) , 
    where D=deg(N,p) and each a_k < p^(p^L).
    This is well-defined and unique by euclidean algorithm.
    '''
    if degree == -1:
        degree = deg(N,p)
    if p == 2:
        power = 1 << (1 << degree)
        decomp = {'prime':p,'degree':degree,0:N % power,1:N // power}
        if as_array:
            return [decomp[k] for k in range(p)] 
        return decomp
    else:
        power = p ** (p ** degree)
        decomp = {'prime':p,'degree':degree,0:N % power}
        for k in range(1,p):
            N = N // power
            decomp[k] = (N % power)
        if as_array:
            return [decomp[k] for k in range(p)]
        return decomp
    
def p_combine(decomp : dict) -> int:
    '''
    the inverse of p_decomp
    '''
    p = decomp['prime']
    degree = decomp['degree']
    if p == 2:
        return decomp[0] + (decomp[1] << (1 << degree))
    else:
        power = p ** (p ** degree)
        N = decomp[0]
        for k in range(1,p):
            N += decomp[k] * power
            power *= p ** (p ** degree)
        return N

In [ ]:
# sanity check
p = 3
N = 100
k = 1256
for n in range(N):
    decomp = p_decomp(k*n,p,as_array=False)
    decomp_array = p_decomp(k*n,p,as_array=True)
    print(f'{k*n} = {decomp}\n = {decomp_array}')
    print(f'{k*n} = {p_combine(decomp)}')

In [88]:
def int_log(N : int, base : int) -> int:
    if N < base:
        return 0
    # for the highest power 'exp' s.t. base^exp <= N, find
    # the largest power of 2 that is less than or equal to exp
    total = 1 << deg(N, base)
    # now divide N to recursively find the binary expansion of exp
    N = N // (base ** total)
    while N != 0:
        total += 1 << deg(N, base)
        N = N // (base ** (1 << deg(N, base)))
    return total - 1

def nim_sum(N : int, M: int, p : int = 2) -> int:
    '''
    Returns the p-nimber sum of N and M
    '''
    # trivial cases
    if N == 0:
        return M
    elif M == 0:
        return N
    # if p = 2, then the sum is just the bitwise xor
    if p == 2:
        return N ^ M
    # otherwise, we do sum modulo p in each "p-bit" 
    if N < p and M < p:
        return (N + M) % p
    else:
        return ((N + M) % p) + p * nim_sum(N//p, M//p,p)

In [84]:
# special case of pi(D):=p**(p**D - 1) is easier to multiply
# because its decomp is [0,0,0,...,0,pi(D-1)]
# 
def pi(degree : int, p : int = 2) -> int:
    if degree == 0:
        return 1
    return p ** (p ** degree - 1)



[0, 0, 0, 0, 625]

In [90]:
# encode p_nimber multiplication recursively via matrices
import numpy as np

def pi_mult(N: int, D : int, p : int = 2) -> int:
    '''returns p**(p**D - 1) * N   w.r.t p-nimber multiplication'''
    if D == 0:
        return N
    decomp_N = p_decomp(N,p,as_array=True,degree = D-1)
    decomp_product = {'prime':p,'degree':D-1}
    for k in range(p):
        decomp_product[k] = 0
    for k in range(p):
        for j in range(p):
            term1 = int((k == p-1) and (j == 0)) * pi_mult(decomp_N[j],D-1,p)
            term2 = int((k > 0) and (j == k)) * pi_mult(decomp_N[j],D-1,p)
            term3 = int(j == k+1) * pi_mult(pi_mult(decomp_N[j],D-1,p),D-1,p)
            sum = nim_sum(term3, nim_sum(term1,term2,p), p)
            decomp_product[k] =  nim_sum(decomp_product[k], sum, p)
    return p_combine(decomp_product)
    

In [92]:
pi_mult(2,1,2)

3